In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("marketing_AB.csv",sep=",")
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


In [4]:
df.drop(df.columns[0], axis=1, inplace=True)
df.head()


,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


### user id - User ID (unique);
### test group - Если "ad", то человек увидел рекламу, если "psa", то он увидел только объявление о государственной службе;
### converted - Если человек купил продукт, то это правда, все остальное - ложь;
### total ads - Количество объявлений, просмотренных одним человеком;
### most ads day - День, когда пользователь увидел наибольшее количество объявлений;
### most ads hour - Час, когда пользователь увидел наибольшее количество объявлений;

## Переводим все в числовые значения

In [5]:
df['most ads day'] = df['most ads day'].map({
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
})
df['converted'] = df['converted'].astype(int)
df['test group'] = df['test group'].map({
    'ad':1,
    'psa':0
})
converted = df.pop('converted') # Перемещаем столбец 'converted' в конец
df['converted'] = converted

In [6]:
df.head(10)

,user id,test group,total ads,most ads day,most ads hour,converted
0,1069124,1,130,0,20,0
1,1119715,1,93,1,22,0
2,1144181,1,21,1,18,0
3,1435133,1,355,1,10,0
4,1015700,1,276,4,14,0
5,1137664,1,734,5,10,0
6,1116205,1,264,2,13,0
7,1496843,1,17,6,18,0
8,1448851,1,21,1,19,0
9,1446284,1,142,0,14,0


### User - id никак не влияет на линейную регрессию

In [7]:
df = df.drop(columns=['user id'])

In [8]:
df.head(10)

,test group,total ads,most ads day,most ads hour,converted
0,1,130,0,20,0
1,1,93,1,22,0
2,1,21,1,18,0
3,1,355,1,10,0
4,1,276,4,14,0
5,1,734,5,10,0
6,1,264,2,13,0
7,1,17,6,18,0
8,1,21,1,19,0
9,1,142,0,14,0


#### Есть ли мультиколинеарность?

In [9]:
df.corr()

,test group,total ads,most ads day,most ads hour,converted
test group,1.000000,0.000279,0.007424,0.006930,0.009611
total ads,0.000279,1.000000,0.002996,-0.010837,0.217419
most ads day,0.007424,0.002996,1.000000,0.014385,-0.020702
most ads hour,0.006930,-0.010837,0.014385,1.000000,0.019674
converted,0.009611,0.217419,-0.020702,0.019674,1.000000


#### Все нормально

### Есть ли в остатках автокоррелированность?

In [10]:
df

,test group,total ads,most ads day,most ads hour,converted
0,1,130,0,20,0
1,1,93,1,22,0
2,1,21,1,18,0
3,1,355,1,10,0
4,1,276,4,14,0
...,...,...,...,...,...
588096,1,1,1,23,0
588097,1,1,1,23,0
588098,1,3,1,23,0
588099,1,1,1,23,0


In [11]:
df.info() # нет ли str переменных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   test group     588101 non-null  int64
 1   total ads      588101 non-null  int64
 2   most ads day   588101 non-null  int64
 3   most ads hour  588101 non-null  int64
 4   converted      588101 non-null  int32
dtypes: int32(1), int64(4)
memory usage: 20.2 MB


In [12]:
import statsmodels.api as sm

# Определяем независимые переменные (exog) и зависимую переменную (endog)
exog = df[['test group', 'total ads', 'most ads day', 'most ads hour']]
endog = df['converted']
exog = sm.add_constant(exog)
model = sm.OLS(endog, exog)

In [13]:
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              converted   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     7465.
Date:                Sat, 01 Jun 2024   Prob (F-statistic):               0.00
Time:                        22:39:10   Log-Likelihood:             2.6952e+05
No. Observations:              588101   AIC:                        -5.390e+05
Df Residuals:                  588096   BIC:                        -5.390e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.0068      0.001     -5.671

### Значение Durbin-Watson:1.993 около 2, это значит что автокоррелированности в остатках нет. 

#### Тест на гетероскедастичность:

In [14]:
from statsmodels.stats.diagnostic import het_breuschpagan
bp_test_statistic, bp_p_value, _, _ = het_breuschpagan(results.resid, exog)

In [15]:
if bp_p_value < 0.05:
    print('Гипотеза о гомоскедастичности отвергается')
else: 
    print('Плаки-плаки')

Гипотеза о гомоскедастичности отвергается


In [16]:
res = model.fit(cov_type='HC3')
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              converted   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     1038.
Date:                Sat, 01 Jun 2024   Prob (F-statistic):               0.00
Time:                        22:39:10   Log-Likelihood:             2.6952e+05
No. Observations:              588101   AIC:                        -5.390e+05
Df Residuals:                  588096   BIC:                        -5.390e+05
Df Model:                           4                                         
Covariance Type:                  HC3                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.0068      0.001     -6.329

### Так как pvalue примерно равно 0, то это значит, что существуют статистически значимые различия между группами control и test